## Curlew Sandpipers at the Menindee lakes

The Menindee Lakes is a chain of shallow ephemeral freshwater lakes connected to the Darling River to form a storage system. The lakes lie in the far west region of New South Wales, Australia, near the town of Menindee.

<img src="data/Menindee_Satellite.png" alt="drawing" width="400" align="left"/>

In the past few years, the Darling River System along with the Menindee Lakes have been severely impacted by extreme drought conditions. This has resulted in particularly low and prolonged storage inflows, lower storage releases, and high evaporation rates. [Source: MDBA](https://www.mdba.gov.au/river-murray-system/running-river-murray/menindee-lakes-facts)

The Menindee Lakes are an integral breeding ground for the hundreds bird, fish and reptile species in the Murray-Darling Basin, one of the richest ecologies in the country.

Lake Menindee has completely dried up several times in the past few years. (Source: ABC News)

<img src="data/menindee_drought.jpg" alt="drawing" width="400" align="left"/>

### Your task:

You are an ecologist working for Wildaroo, a non-for-profit organisation that protects endangered species across Australia. You have been tasked with drafting an action plan to protect the Curlew Sandpipers, a species of birds commonly found in the Menindee lakes. The number of birds spotted has been declining in recent years due to the lack of water in the region. Wildaroo wants to start an intervention to increase the numbers of Sandpipers in the area.

Your mission is to identify the areas that are most affected by drought and decide the best locations where your organisation should deploy birdfeeders to support the existing population. The ideal place should be close to wet areas, which is the natural habitat of Curlew Sandpipers.

### Load packages

You start by loading the usual Python libraries to start working on this project.

In [1]:
%matplotlib inline

import datacube

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

### Load data

You request Water Observations from Space (WOfS) data for the first 6 months of the year.

In [2]:
dc_menindee = datacube.Datacube(app="Menindee_Lakes")

query = {'lat': (-32.55, -32.25),
         'lon': (142.15, 142.45),
         'time':('2019-01-01', '2019-06-01')}


menindee_19 = dc_menindee.load(product='wofs_albers', **query)

menindee_19

<xarray.Dataset>
Dimensions:      (time: 72, x: 1225, y: 1428)
Coordinates:
  * time         (time) datetime64[ns] 2019-01-01T00:20:16 ... 2019-06-01T00:...
  * y            (y) float64 -3.553e+06 -3.553e+06 ... -3.589e+06 -3.589e+06
  * x            (x) float64 9.447e+05 9.447e+05 ... 9.753e+05 9.753e+05
    spatial_ref  int32 3577
Data variables:
    water        (time, y, x) int16 0 0 0 0 0 0 0 ... 239 239 239 239 239 239
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

### Interpreting WOfS

You want to understand what values are contained in this dataset and use `unique`, a numpy function that returns the unique values contained in an array:

In [ ]:
np.unique(menindee_19.water.data)

There is a special function in DEA that displays the interpretation of mask bitflags in collection. So, you import the required library from the Datacube.

In [ ]:
from datacube.utils import masking

masking.describe_variable_flags(menindee_19, with_pandas=True)

### Creating the mask and computing the median water content

Wet areas are represented using the value `128`, as oposed to dry areas which are represented with the `0` value. You represent the median water content over the first 6 months.

In [ ]:
m2019 = menindee_19.water.where(menindee_19.water == 128).median(dim='time')
m2019.plot()

### Replicate for 2001

In [ ]:
query = {'lat': (-32.55, -32.25),
         'lon': (142.15, 142.45),
         'time':('2001-01-01', '2001-06-01')}


menindee_01 = dc_menindee.load(product='wofs_albers', **query)

menindee_01

In [ ]:
m2001 = menindee_01.water.where(menindee_01.water == 128).median(dim='time')
m2001.plot()

### Representing changes in water using a Semaphore plot

You want to create a plot to represent changes in the water between the years 2001 and 2019. The plot maps the intersection of water areas of both years with the following mapping:

* If a pixel  contains water in both years -> Blue
* If a pixel had water before but doesn't have now -> Red (water lost)
* If a pixel didn't have water but has water now -> Green (new water)
* If a pixel never had water -> Transparent (RGB supports a 4th channel called Alpha for transparency)

_Hint: This is an example of what a semaphore plot looks like for the years 2016-2019 at this location:_
<img src="data/semaphore.png" alt="drawing" width="400"/>

### Coding the semaphore plot

For creating this plot you'll need to:

1. Transform `m2001` and `m2019` to boolean types (water/dry) using a threshold.
2. Use numpy logical functions to compute the boolean values of the different components Red, Green Blue, Alpha. _int: use `np.logical_not()` and `np.logical_and()` to compute the components.
3. Stack the components along a 3rd dimension making use of `np.dstack`
4. Convert the resulting array into `float64` type
5. Plot using matplotlib

In [ ]:
### Your code goes here

### Interpreting the result

This figure is quite striking and gives you an idea about why the Curlew Sandpipers population has such low numbers in recent years. However, you would like to do some more comparisons with more recent years at different seasons to understand the variability and representativeness of these results. 